In [1]:
import pandas as pd
import sqlite3
import numpy as np
from itertools import product
pd.set_option("display.max_columns", 500)

In [2]:
data = pd.read_csv('training_data.csv')

In [3]:
countries = data.loc[:, 'Area'].unique()

In [4]:
gdp = pd.read_excel('countries_gdp.xlsx')
pop = pd.read_excel('population_growth.xlsx')

gdp.sort_values(by='country', inplace=True)
pop.sort_values(by='COUNTRY', inplace=True)

In [5]:
if input("load maps? y/n") == 'y':
    gdp = pd.read_csv('gdp_mapped.csv')
    pop = pd.read_csv('pop_mapped.csv')

load maps? y/ny


In [6]:
if input("create new maps? y/n") == 'y':
    gdp_map = {}
    for c in gdp.itertuples():
        if c.country in countries:
            gdp_map[c.country] = c.country
        else:
            while gdp_map.get(c.country) not in countries and gdp_map.get(c.country) != 'nan':
                gdp_map[c.country] = input(c.country)    
    gdp['mapped'] = gdp['country'].apply(lambda x: gdp_map.get(x, 'nan'))
    gdp = gdp.loc[gdp.loc[:,'mapped']!='nan',:]
    
    pop_map = {}
    for c in pop.itertuples():
        if c.COUNTRY in countries:
            pop_map[c.COUNTRY] = c.COUNTRY
        else:
            while pop_map.get(c.COUNTRY) not in countries and pop_map.get(c.COUNTRY) != 'nan':
                pop_map[c.COUNTRY] = input(c.COUNTRY)
    pop['mapped'] = pop['COUNTRY'].apply(lambda x: pop_map.get(x, 'nan'))
    pop = pop.loc[pop.loc[:,'mapped']!='nan',:]          

create new maps? y/nn


In [7]:
if input("save maps again? y/n") == 'y':
    gdp.to_csv('gdp_mapped.csv', index=False)
    pop.to_csv('pop_mapped.csv', index=False)

save maps again? y/nn


In [8]:
pop.shape, gdp.shape, len(countries)

((177, 3), (174, 3), 167)

In [10]:
pop.set_index('mapped', inplace=True)
gdp.set_index('mapped', inplace=True)

In [11]:
pop['POPULATION'] = pop['POPULATION'].apply(lambda x: float("-" + x[1:]) if "−" in str(x) else x)

In [12]:
years = np.concatenate([data['YearBin'].unique(),[2020, 2025, 2030]])

In [13]:
def forecast(start, rate, years):
    return start*(1+rate/100)**years

In [14]:
new_data = pd.DataFrame(list(product(countries, years)), columns = ['Area', 'YearBin'])
new_data.set_index(['Area', 'YearBin'], inplace=True)
new_data = new_data.join(data.set_index(['Area', 'YearBin']), how='left')

In [15]:
All = slice(None)
col_pop = 'Total population'
for name, group in new_data.groupby('Area'):
    last_val = group.at[(name, 2015), col_pop]
    rate = pop.at[name, 'POPULATION']
    new_data.at[(name, 2020), col_pop] = forecast(last_val, rate, 5)
    new_data.at[(name, 2025), col_pop] = forecast(last_val, rate, 10)
    new_data.at[(name, 2030), col_pop] = forecast(last_val, rate, 15)

In [16]:
col_gdp = 'Gross Domestic Product (GDP)'
world_gdp = 3.2
for name, group in new_data.groupby('Area'):
    last_val = group.at[(name, 2015), col_gdp]
    if name in gdp.index:
        rate = gdp.at[name, 'gdp_rate']
    else:
        print(name)
        rate = world_gdp
    new_data.at[(name, 2020), col_gdp] = forecast(last_val, rate, 5)
    new_data.at[(name, 2025), col_gdp] = forecast(last_val, rate, 10)
    new_data.at[(name, 2030), col_gdp] = forecast(last_val, rate, 15)

Bhutan
Cuba
Syrian Arab Republic


In [18]:
new_data

Total population  \
Area        YearBin                     
Afghanistan 1980         12750.000000   
            1985         11503.000000   
            1990         13981.000000   
            1995         18382.000000   
            2000         21980.000000   
            2005         26617.000000   
            2010         30697.000000   
            2015         35530.000000   
            2020         40022.760330   
            2025         45083.629170   
            2030         50784.443711   
Albania     1980          2788.000000   
            1985          3121.000000   
            1990          3241.000000   
            1995          3093.000000   
            2000          3119.000000   
            2005          3024.000000   
            2010          2920.000000   
            2015          2930.000000   
            2020          2949.094581   
            2025          2968.313601   
            2030          2987.657869   
Algeria     1980         20576.000000   
            1985         23918.000000   
            1990         27181.000000   
            1995         29887.000000   
            2000         31995.000000   
            2005         34300.000000   
            2010         37566.000000   
            2015         41318.000000   
...                               ...   
Yemen       1995         16409.000000   
            2000         18919.000000   
            2005         21752.000000   
            2010         24910.000000   
            2015         28250.000000   
            2020         31698.106687   
            2025         35567.078498   
            2030         39908.284912   
Zambia      1980          6306.000000   
            1985          7386.000000   
            1990          8452.000000   
            1995          9667.000000   
            2000         11120.000000   
            2005         12726.000000   
            2010         14700.000000   
            2015         17094.000000   
            2020         19787.788661   
            2025         22906.082841   
            2030         26515.779005   
Zimbabwe    1980          7722.000000   
            1985          9293.000000   
            1990         10683.000000   
            1995         11710.000000   
            2000         12501.000000   
            2005         13330.000000   
            2010         14711.000000   
            2015         16530.000000   
            2020         18502.331182   
            2025         20709.997531   
            2030         23181.078833   

                     Cultivated area (arable land + permanent crops)  \
Area        YearBin                                                    
Afghanistan 1980                                              8054.0   
            1985                                              8045.0   
            1990                                              8030.0   
            1995                                              7790.0   
            2000                                              7753.0   
            2005                                              7910.0   
            2010                                              7910.0   
            2015                                              7910.0   
            2020                                                 NaN   
            2025                                                 NaN   
            2030                                                 NaN   
Albania     1980                                               709.0   
            1985                                               714.0   
            1990                                               703.0   
            1995                                               700.0   
            2000                                               699.0   
            2005                                               698.0   
            2010                                               696.0 

In [19]:
new_data.to_csv('training_data_rate.csv')